# support-queue-assistant-rag

In [1]:
# Uncomment and execute this once to download minsearch
# !wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py

In [2]:
# Import libraries
import pandas as pd
import random
import json
from tqdm.auto import tqdm
import minsearch
from openai import OpenAI

## Ingestion

In [3]:
df = pd.read_csv('../data/data-clean.csv', sep=",")
df.head()

,id,question,category,subcategory,intent,product_feature,response,related_articles,tags
0,0,How do I reset my password?,Account,Password Management,Password Recovery,Authentication & Security,"To reset your password, click on the 'Forgot P...",https://help.example.com/account/reset-password,"password, reset, account access"
1,1,What payment methods do you accept?,Billing,Payment Options,Payment Information,Billing & Invoicing,"We accept major credit cards (Visa, MasterCard...",https://help.example.com/billing/payment-methods,"payment, credit card, PayPal"
2,2,Can I integrate your service with Salesforce?,Technical,Integrations,Integration Setup,API & Integrations,"Yes, we offer a Salesforce integration. You ca...",https://developers.example.com/integrations/sa...,"Salesforce, integration, API"
3,3,How do I add a new user to my account?,Account,User Management,Add User,User Roles & Permissions,"To add a new user, go to 'Settings' > 'User Ma...",https://help.example.com/account/adding-users,"add user, invite, user management"
4,4,What's included in the Enterprise plan?,General Inquiry,Pricing & Plans,Plan Information,Subscription Management,"Our Enterprise plan includes unlimited users, ...",https://www.example.com/pricing,"Enterprise plan, pricing, features"


In [5]:
df.shape

(125, 9)

In [6]:
df.columns

Index(['id', 'question', 'category', 'subcategory', 'intent',
       'product_feature', 'response', 'related_articles', 'tags'],
      dtype='object')

In [7]:
documents = df.to_dict(orient='records')

In [8]:
documents[0]

{'id': 0,
 'question': 'How do I reset my password?',
 'category': 'Account',
 'subcategory': 'Password Management',
 'intent': 'Password Recovery',
 'product_feature': 'Authentication & Security',
 'response': "To reset your password, click on the 'Forgot Password' link on the login page. Enter your email address, and we'll send you instructions to create a new password.",
 'related_articles': 'https://help.example.com/account/reset-password',
 'tags': 'password, reset, account access'}

In [9]:
index = minsearch.Index(
    text_fields=['question', 'category', 'subcategory', 'intent',
       'product_feature', 'response', 'related_articles', 'tags'],
    keyword_fields=['id']
)

In [10]:
index.fit(documents)

In [11]:
query = 'How do I integrate your API with my CRM?'

In [12]:
index.search(query, num_results=10)

[{'id': 12,
  'question': 'What APIs do you offer?',
  'category': 'Technical',
  'subcategory': 'API',
  'intent': 'API Information',
  'product_feature': 'API & Integrations',
  'response': 'We offer RESTful APIs for most of our services, including data retrieval, user management, and reporting. Full documentation, including endpoints and example requests, can be found in our API documentation.',
  'related_articles': 'https://developers.example.com/api',
  'tags': 'API, integration, development'},
 {'id': 41,
  'question': 'How do I generate an API token?',
  'category': 'Technical',
  'subcategory': 'API',
  'intent': 'Generate API Token',
  'product_feature': 'API & Integrations',
  'response': "To generate an API token, go to 'Settings' > 'API & Integrations' > 'API Tokens'. Click 'Generate New Token', give it a name and set the expiration. Copy the token immediately as it won't be displayed again. Remember to keep your tokens secure.",
  'related_articles': 'https://developers.e

## RAG Flow

In [13]:
client = OpenAI(api_key = 'YOUR_KEY')

In [14]:
def search(query):
    boost = {}

    results = index.search(
        query=query,
        filter_dict={},
        boost_dict=boost,
        num_results=10
    )

    return results

In [15]:
prompt_template = """
You're a B2B SaaS company's customer support analyst. Answer the QUESTION based on the CONTEXT from our support queue knowledge database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

entry_template = """
question: {question}
category: {category}
subcategory: {subcategory}
intent: {intent}
product_feature: {product_feature}
response: {response}
related_articles: {related_articles}
tags: {tags}
""".strip()

def build_prompt(query, search_results):
    context = ""
    
    for doc in search_results:
        context = context + entry_template.format(**doc) + "\n\n"

    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [16]:
def llm(prompt, model='gpt-4o-mini'):
    response = client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [17]:
def rag(query, model='gpt-4o-mini'):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    #print(prompt)
    answer = llm(prompt, model=model)
    return answer

In [18]:
question = 'How can I reset my password?'
answer = rag(question)
print(answer)

To reset your password, click on the 'Forgot Password' link on the login page. Enter your email address, and we'll send you instructions to create a new password. For more details, you can also visit our help article: [Reset Password](https://help.example.com/account/reset-password).


In [19]:
question = 'How can I setup enterprise account?'
answer = rag(question)
print(answer)

To set up an enterprise account, you should contact our sales team to discuss your organization's size and needs. They will help you create a tailored enterprise license proposal, which typically includes unlimited users, priority support, and custom terms. For more information, visit [this link](https://www.example.com/enterprise-licensing).


## Retrieval evaluation

Make sure that ground truth data was generated using `evaluation-data-generation.ipynb`

In [20]:
df_question = pd.read_csv('../data/ground-truth-retrieval.csv')

In [21]:
df_question

,id,question
0,0,What steps do I need to follow to reset my pas...
1,0,Can you guide me through the password recovery...
2,0,Where can I find the link to reset my password...
3,0,Is there a specific email address I need to us...
4,0,What resources are available if I encounter is...
...,...,...
620,124,What are the steps to create a custom workflow...
621,124,Can I integrate external systems when setting ...
622,124,How does the Workflow Builder help in automati...
623,124,Is there a way to test my custom workflow befo...


In [22]:
ground_truth = df_question.to_dict(orient='records')

In [23]:
ground_truth[0]

{'id': 0,
 'question': 'What steps do I need to follow to reset my password for my account?'}

In [24]:
def hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)

def mrr(relevance_total):
    total_score = 0.0

    for line in relevance_total:
        for rank in range(len(line)):
            if line[rank] == True:
                total_score = total_score + 1 / (rank + 1)

    return total_score / len(relevance_total)

In [25]:
def minsearch_search(query):
    boost = {}

    results = index.search(
        query=query,
        filter_dict={},
        boost_dict=boost,
        num_results=10
    )

    return results

In [26]:
def evaluate(ground_truth, search_function):
    relevance_total = []

    for q in tqdm(ground_truth):
        doc_id = q['id']
        results = search_function(q)
        relevance = [d['id'] == doc_id for d in results]
        relevance_total.append(relevance)

    return {
        'hit_rate': hit_rate(relevance_total),
        'mrr': mrr(relevance_total),
    }

In [27]:
evaluate(ground_truth, lambda q: minsearch_search(q['question']))

  0%|          | 0/625 [00:00<?, ?it/s]

{'hit_rate': 0.968, 'mrr': 0.834332698412698}

## Finding the best parameters

In [29]:
df_validation = df_question[:60]
df_test = df_question[60:]

In [33]:
df_validation.shape, df_test.shape

((60, 2), (565, 2))

In [31]:
df_test

,id,question
60,12,What types of APIs does your company provide?
61,12,Can you explain the capabilities of your APIs ...
62,12,Where can I find the documentation for your APIs?
63,12,Do your APIs support user management functiona...
64,12,Are example requests included in your API docu...
...,...,...
620,124,What are the steps to create a custom workflow...
621,124,Can I integrate external systems when setting ...
622,124,How does the Workflow Builder help in automati...
623,124,Is there a way to test my custom workflow befo...


In [34]:
import random

def simple_optimize(param_ranges, objective_function, n_iterations=10):
    best_params = None
    best_score = float('-inf')  # Assuming we're minimizing. Use float('-inf') if maximizing.

    for _ in range(n_iterations):
        # Generate random parameters
        current_params = {}
        for param, (min_val, max_val) in param_ranges.items():
            if isinstance(min_val, int) and isinstance(max_val, int):
                current_params[param] = random.randint(min_val, max_val)
            else:
                current_params[param] = random.uniform(min_val, max_val)
        
        # Evaluate the objective function
        current_score = objective_function(current_params)
        
        # Update best if current is better
        if current_score > best_score:  # Change to > if maximizing
            best_score = current_score
            best_params = current_params
    
    return best_params, best_score

In [35]:
gt_val = df_validation.to_dict(orient='records')

In [36]:
def minsearch_search(query, boost=None):
    if boost is None:
        boost = {}

    results = index.search(
        query=query,
        filter_dict={},
        boost_dict=boost,
        num_results=10
    )

    return results

In [37]:
param_ranges = {
    'question': (0.0, 3.0),
    'category': (0.0, 3.0),
    'subcategory': (0.0, 3.0),
    'intent': (0.0, 3.0),
    'product_feature': (0.0, 3.0),
    'response': (0.0, 3.0),
    'related_articles': (0.0, 3.0),
    'tags': (0.0, 3.0),
}

def objective(boost_params):
    def search_function(q):
        return minsearch_search(q['question'], boost_params)

    results = evaluate(gt_val, search_function)
    return results['mrr']

In [38]:
simple_optimize(param_ranges, objective, n_iterations=20)

  0%|          | 0/60 [00:00<?, ?it/s]

  0%|          | 0/60 [00:00<?, ?it/s]

  0%|          | 0/60 [00:00<?, ?it/s]

  0%|          | 0/60 [00:00<?, ?it/s]

  0%|          | 0/60 [00:00<?, ?it/s]

...

  0%|          | 0/60 [00:00<?, ?it/s]

  0%|          | 0/60 [00:00<?, ?it/s]

  0%|          | 0/60 [00:00<?, ?it/s]

  0%|          | 0/60 [00:00<?, ?it/s]

({'question': 2.3190067555885863,
  'category': 0.34225912296078376,
  'subcategory': 1.1971703133905314,
  'intent': 1.5157268043510923,
  'product_feature': 1.115129576114986,
  'response': 2.511416656464337,
  'related_articles': 0.23824890259412224,
  'tags': 1.662920970854158},
 0.8008531746031745)

In [39]:
def minsearch_improved(query):
    boost = {
        'question': 2.32,
        'category': 0.34,
        'subcategory': 1.20,
        'intent': 1.52,
        'product_feature': 1.16,
        'response': 2.51,
        'related_articles': 0.24,
        'tags': 1.66,
    }

    results = index.search(
        query=query,
        filter_dict={},
        boost_dict=boost,
        num_results=10
    )

    return results

evaluate(ground_truth, lambda q: minsearch_improved(q['question']))

  0%|          | 0/625 [00:00<?, ?it/s]

{'hit_rate': 0.9744, 'mrr': 0.8697815873015872}

## RAG evaluation

Let's use LLM-as-a-judge

In [40]:
prompt2_template = """
You are an expert evaluator for a RAG system.
Your task is to analyze the relevance of the generated answer to the given question.
Based on the relevance of the generated answer, you will classify it
as "NON_RELEVANT", "PARTLY_RELEVANT", or "RELEVANT".

Here is the data for evaluation:

Question: {question}
Generated Answer: {answer_llm}

Please analyze the content and context of the generated answer in relation to the question
and provide your evaluation in parsable JSON without using code blocks:

{{
  "Relevance": "NON_RELEVANT" | "PARTLY_RELEVANT" | "RELEVANT",
  "Explanation": "[Provide a brief explanation for your evaluation]"
}}
""".strip()

In [41]:
len(ground_truth)

625

In [42]:
record = ground_truth[0]
question = record['question']
answer_llm = rag(question)

In [43]:
print(answer_llm)

To reset your password, follow these steps:

1. Click on the 'Forgot Password' link on the login page.
2. Enter your email address.
3. Check your email for instructions to create a new password.

For more details, you can refer to the related article: [Password Reset Instructions](https://help.example.com/account/reset-password).


In [44]:
prompt = prompt2_template.format(question=question, answer_llm=answer_llm)
print(prompt)

You are an expert evaluator for a RAG system.
Your task is to analyze the relevance of the generated answer to the given question.
Based on the relevance of the generated answer, you will classify it
as "NON_RELEVANT", "PARTLY_RELEVANT", or "RELEVANT".

Here is the data for evaluation:

Question: What steps do I need to follow to reset my password for my account?
Generated Answer: To reset your password, follow these steps:

1. Click on the 'Forgot Password' link on the login page.
2. Enter your email address.
3. Check your email for instructions to create a new password.

For more details, you can refer to the related article: [Password Reset Instructions](https://help.example.com/account/reset-password).

Please analyze the content and context of the generated answer in relation to the question
and provide your evaluation in parsable JSON without using code blocks:

{
  "Relevance": "NON_RELEVANT" | "PARTLY_RELEVANT" | "RELEVANT",
  "Explanation": "[Provide a brief explanation for yo

In [45]:
llm(prompt)

'{\n  "Relevance": "RELEVANT",\n  "Explanation": "The generated answer accurately provides specific steps necessary to reset a password, directly addressing the question asked about the process to follow for resetting the password."\n}'

In [47]:
df_sample = df_question.sample(n=200, random_state=1)

In [48]:
sample = df_sample.to_dict(orient='records')

In [49]:
evaluations = []

for record in tqdm(sample):
    question = record['question']
    answer_llm = rag(question) 

    prompt = prompt2_template.format(
        question=question,
        answer_llm=answer_llm
    )

    evaluation = llm(prompt)
    evaluation = json.loads(evaluation)

    evaluations.append((record, answer_llm, evaluation))

  0%|          | 0/200 [00:00<?, ?it/s]

In [50]:
evaluations[0]

({'id': 110,
  'question': 'What is the best way to manage tenant provisioning and data separation?'},
 'The best way to manage tenant provisioning and data separation is to implement a multi-tenant architecture using our Multi-Tenant SDK. This involves designing your data model with tenant isolation, which can be achieved through either separate databases or row-level security. The SDK will assist you in managing tenant provisioning, ensuring data separation, and facilitating cross-tenant operations. Additionally, you can configure tenant-specific branding and settings within the admin panel to further customize the experience for each tenant. For more detailed guidance, you can refer to our documentation on implementing multi-tenant architecture.',
 {'Relevance': 'RELEVANT',
  'Explanation': 'The generated answer directly addresses the question about managing tenant provisioning and data separation by discussing the implementation of a multi-tenant architecture, tenant isolation meth

In [51]:
df_eval = pd.DataFrame(evaluations, columns=['record', 'answer', 'evaluation'])

df_eval['id'] = df_eval.record.apply(lambda d: d['id'])
df_eval['question'] = df_eval.record.apply(lambda d: d['question'])

df_eval['relevance'] = df_eval.evaluation.apply(lambda d: d['Relevance'])
df_eval['explanation'] = df_eval.evaluation.apply(lambda d: d['Explanation'])

del df_eval['record']
del df_eval['evaluation']

In [58]:
df_eval.relevance.value_counts()

relevance
RELEVANT           188
PARTLY_RELEVANT     11
NON_RELEVANT         1
Name: count, dtype: int64

In [53]:
df_eval.relevance.value_counts(normalize=True)

relevance
RELEVANT           0.940
PARTLY_RELEVANT    0.055
NON_RELEVANT       0.005
Name: proportion, dtype: float64

In [54]:
df_eval.to_csv('../data/rag-eval-gpt-4o-mini.csv', index=False)

In [56]:
df_eval[df_eval.relevance == 'PARTLY_RELEVANT']

,answer,id,question,relevance,explanation
10,"Before performing penetration testing, you nee...",98,What are the terms I need to agree to before p...,PARTLY_RELEVANT,The generated answer touches on the need to no...
43,The specific programming languages supported b...,54,What programming languages are supported by yo...,PARTLY_RELEVANT,The generated answer acknowledges that various...
45,To configure scopes and attribute mapping when...,75,What should I configure for scopes and attribu...,PARTLY_RELEVANT,The generated answer touches on configuring a ...
100,The provided context does not contain informat...,113,Is there a way to automate the fulfillment of ...,PARTLY_RELEVANT,The generated answer acknowledges that there i...
105,"Based on the provided context, there is no spe...",121,Is it possible to implement k-anonymity algori...,PARTLY_RELEVANT,The generated answer acknowledges the lack of ...
111,"Based on the provided context, there is no spe...",114,Is there a way to customize dashboards program...,PARTLY_RELEVANT,The generated answer acknowledges the lack of ...
124,To contact the sales team for an enterprise li...,78,How do I contact the sales team for an enterpr...,PARTLY_RELEVANT,The answer provides some guidance on contactin...
136,"To optimize algorithms for energy efficiency, ...",80,How do you optimize your algorithms for energy...,PARTLY_RELEVANT,The generated answer discusses energy efficien...
143,"No, the provided context does not mention whet...",72,Is a Business Associate Agreement included in ...,PARTLY_RELEVANT,The generated answer addresses the question by...
144,The default location for data storage is in ou...,60,What is the default location for data storage?,PARTLY_RELEVANT,The generated answer mentions the default loca...


In [57]:
df_eval[df_eval.relevance == 'NON_RELEVANT']

,answer,id,question,relevance,explanation
132,The provided context does not contain any info...,90,What is your TLS version for securing data in ...,NON_RELEVANT,The generated answer states that there is no i...


In [59]:
evaluations_gpt4o = []

for record in tqdm(sample):
    question = record['question']
    answer_llm = rag(question, model='gpt-4o') 

    prompt = prompt2_template.format(
        question=question,
        answer_llm=answer_llm
    )

    evaluation = llm(prompt)
    evaluation = json.loads(evaluation)
    
    evaluations_gpt4o.append((record, answer_llm, evaluation))

  0%|          | 0/200 [00:00<?, ?it/s]

In [60]:
evaluations_gpt4o[0]

({'id': 110,
  'question': 'What is the best way to manage tenant provisioning and data separation?'},
 'The best way to manage tenant provisioning and data separation in a multi-tenant architecture is to use our Multi-Tenant SDK. Here are the essential steps:\n\n1. **Design Your Data Model**: Ensure tenant isolation by using either separate databases or row-level security mechanisms.\n2. **Use the Multi-Tenant SDK**: This SDK aids in tenant provisioning, data separation, and handling cross-tenant operations.\n3. **Tenant-Specific Configurations**: Customize tenant-specific branding and settings via the admin panel.\n\nBy following these steps, you can efficiently manage tenant provisioning and maintain strong data separation between tenants.\n\nFor more detailed guidance, refer to our [Developer Documentation on Multi-Tenant SDK](https://developers.example.com/multi-tenant-sdk).',
 {'Relevance': 'RELEVANT',
  'Explanation': 'The generated answer directly addresses the question about m

In [61]:
df_eval = pd.DataFrame(evaluations_gpt4o, columns=['record', 'answer', 'evaluation'])

df_eval['id'] = df_eval.record.apply(lambda d: d['id'])
df_eval['question'] = df_eval.record.apply(lambda d: d['question'])

df_eval['relevance'] = df_eval.evaluation.apply(lambda d: d['Relevance'])
df_eval['explanation'] = df_eval.evaluation.apply(lambda d: d['Explanation'])

del df_eval['record']
del df_eval['evaluation']

In [62]:
df_eval.relevance.value_counts()

relevance
RELEVANT           189
PARTLY_RELEVANT     11
Name: count, dtype: int64

In [63]:
df_eval.relevance.value_counts(normalize=True)

relevance
RELEVANT           0.945
PARTLY_RELEVANT    0.055
Name: proportion, dtype: float64

In [64]:
df_eval.to_csv('../data/rag-eval-gpt-4o.csv', index=False)